# Обучение с учителем
## Применение линейной регрессии

В этом задании вам предлагается изучить и применить на практике модели линейной регресии,
доступные в библиотеке `scikit-learn`.

Модель линейной регрессии имеет ряд проблем, связанных с переобучением. Другими словами, модель слишком точно приближает зависимость между признаками и целевой переменной на тренировочной выборке, и, как результат, делает достаточно грубые или неточные предсказания на новых данных. Регуляризация является эффективным решением данной проблемы. Введем понятие Гребневой Регрессии (Ridge Regression).

Сформулируем задачу минимизации функционала $Q(\beta)$ для нахождения линейной зависимости между целевой переменной $Y$ и признаками.

$$
\begin{equation*}
    Q(\beta) = \| Y - X \beta \|^2 + \lambda \| \beta \|^2 \rightarrow \min\limits_{\beta}.
\end{equation*}
$$

В данном случае, $X$ - матрица признаков, $\beta$ - вектор параметров, который требуется найти. Таким образом, мы добавляем к функции потерь регуляризатор с параметром $\lambda$, штрафующий большие значения вектора весов $\beta$. 

Настройка коэффициента регуляризации $\lambda$ - это важная задача в построении модели Гребневой регрессии.

## Загрузка и разбиение данных

Мы будем решать задачу линейной регресии на наборе данных `Boston House Prices`. Он представляет из себя набор различных факторов, которые некоторым образов влияют на среднюю рыночную цену дома в разных районах Бостона. Например, уровень преступности или среднее количество комнат. Наша задача - предсказать цену нового объекта, опираясь на значения этих признаков.

Начнем с загрузки этого набора данных и его разбиения. `Boston House Prices` является одним из втроенных наборов данных, доступных в `scikit-learn`. Для его загрузки достаточно вызвать функцию `load_boston` из `sklearn.datasets`. Далее, предлагается случайно разбить выборку на тренировочную и тестовую выборку, используя функцию `train_test_split` из `sklearn.model_selection`, вызвав ее с параметром `random_state=54` и `test_size=0.33`.

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston

boston = load_boston()
X_train, X_test, y_train, y_test = train_test_split(boston.data, boston.target, random_state=54, test_size=0.33)

## Настройка параметра регуляризации в Гребневой регрессии

Модель Гребневой регрессии представлена классом `Ridge` из `sklearn.linear_model`. Конструктор этого класса содержит аргумент `alpha`, обратное значение которого соответствует параметру регуляризации $\lambda$. При использовании данной модели установите значение аргумента `random_state=42`. Найдите оптимальное значение коэффициента `alpha` в диапазоне `{10, 20, 30, ..., 90}` начиная со значения 10 до 90 с шагом 10, соответствующее минимальному значению среднеквадратической ошибки на тестовой выборке. Так же укажите это минимальное значение, округленное до трех знаков после запятой. Эти два значения будут являться первым `answer1` и вторым `answer2` ответом в этом практическом задание. Постройте график зависимости среднеквадратической ошибки от значения параметра `alpha`.

### *РЕШЕНИЕ*

In [4]:
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot as plt

In [12]:
# Создаем список содержащий проверяемые значения коэфициента alpha
value_alpha = [i for i in range(10,91,10)]
# Список где будут хранится найденые значения минимальной среднеквадратической ошибки
mse_errors = {}

In [13]:
# В цикле создаем обьекты класса Ridge с различными коэфициентами alhpa
for alpha in value_alpha:
    ridge = Ridge(alpha=alpha,random_state=42)
    # Тренируем модель
    ridge.fit(X_train,y_train)
    # Предсказываем 
    predictions = ridge.predict(X_test)
    value_mse = mean_squared_error(y_test,predictions)
    mse_errors[alpha] = value_mse


In [14]:
# Сортируем по значению mse
# Превращаем словарь в список кортежей(где первый элемент это ключ а второй значение)
mse_list = list(mse_errors.items())
# сортируем по возрастанию(ascending)
mse_list.sort(key=lambda x:x[1])
mse_list

[(50, 23.131846027634133),
 (40, 23.14690389145796),
 (60, 23.149749882502075),
 (70, 23.1899770201459),
 (30, 23.21072612489941),
 (80, 23.245318043037354),
 (90, 23.31082865744054),
 (20, 23.346202568579088),
 (10, 23.577406074332597)]

In [15]:
answer1, answer2 = mse_list[0][0],mse_list[0][1]
answer1,answer2

(50, 23.131846027634133)

In [ ]:
# построение графика
import pandas as pd
graph = pd.DataFrame(mse_errors)

В результате, мы нашли оптимальное значение коэффициента `alpha` из предложенных, которое соответствует наиболее точной модели на новых данных. Таким образом, Гребневая регрессия собирает максимум информации из предложенных тренировочных данных.

## Метод регрессии LASSO

Еще одним методом регуляризации, похожем на Гребневую регрессию, является LASSO (least absolute shrinkage and selection operator). В данном случае, задача минимизации функции потерь формулируется следующим образом.

$$
\begin{equation*}
    \left\lbrace
        \begin{array}{c}
            Q(\beta) = \| Y - X \beta \|^2, \\
            \sum\limits_{i=0}^N |\beta_i| < \varkappa.
        \end{array}
    \right.
\end{equation*}
$$

При уменьшении значения $\varkappa$ все больше коэффициентов $\beta_i$ обнуляются. Таким образом, LASSO является своеобразным методом отбора признаков и понижения размерности, подробнее о которых вы узнаете на 4-й неделе нашего курса. Этот метод регуляризации выбирает самые информативные признаки, отбрасывая остальные.

Загрузите набор данных `Diabetes`, используя функцию `load_diabetes` из `sklearn.datasets`. `Diabetes` содержит в себе информацию о различных признаках, таких как возраст, пол, среднее кровяное давление человека и прочие. В качестве целевой переменной выступает количественный показатель прогрессирования заболевания диабет через год после определенного периода (baseline).

In [21]:
from sklearn.datasets import load_diabetes

diabetes = load_diabetes()

In [22]:
from sklearn.linear_model import Lasso

In [27]:
diabetes
# Создаем объект класса Lasso
lasso = Lasso(random_state=42)

# Обучаем модель
lasso.fit(diabetes.data, diabetes.target)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=42,
      selection='cyclic', tol=0.0001, warm_start=False)

In [28]:
lasso.coef_

array([  0.        ,  -0.        , 367.70185207,   6.30190419,
         0.        ,   0.        ,  -0.        ,   0.        ,
       307.6057    ,   0.        ])

In [29]:
# Находим отношение выбраных моделей
# Индусский метод
lasso_coef = [x for x in lasso.coef_ if x > 0 ]
lasso_coef

[367.7018520698164, 6.301904186054684, 307.60570000448143]

In [40]:
# Правильный метод
lass_coef = lasso.coef_[lasso.coef_ != 0]
lasso_coef

array([367.70185207,   6.30190419, 307.6057    ])

In [42]:
# Еще один правильный метод
lasso.coef_[np.nonzero(lasso.coef_)]


array([367.70185207,   6.30190419, 307.6057    ])

In [44]:
# Определяем ответ3
answer3 = len(lasso_coef)/len(lasso.coef_)
answer3

0.3

Метод LASSO представлен классом `Lasso` из `sklearn.linear_model`. Обучите эту модель на всем наборе данных `Diabetes`. Выберете в качестве параметра конструктора класса `random_state=42`. Какое количество признаков было отсеяно данной моделью? Для этого проанализируйте вектор весов `coef_`, являющийся атрибутом данной модели. В качестве ответа `answer3` на это задание приведите отношение числа выбранных моделью признаков к их общему числу. Ответ округлите до одного знака после запятой.

### *РЕШЕНИЕ*

In [ ]:
from sklearn.linear_model import Lasso


Далее обучите модель `Lasso` с параметром конструктора `random_state=42` на тренировочной выборке датасета `Boston House Prices` и посчитайте значение среднеквадратической ошибки на тестовой выборке. Приведите это значение в качестве ответа `answer4`, округленного до трех знаков после запятой. Сравните это значение с результатом, полученным Гребневой регрессией.

### *РЕШЕНИЕ*

In [51]:
lasso_boston = Lasso(random_state=42)
# Обучение
lasso_boston.fit(X_train,y_train)
# Предсказание
predictions = lasso_boston.predict(X_test)

# Метрика качества
mse_error = mean_squared_error(y_test,predictions)
answer4 = mse_error

# Строка с ответами

In [52]:
output = "alpha: {0}\nmse error(ridge): {1:.3f}\nratio: {2:.1f}\nmse error(lasso): {3:.3f}"
print(output.format(answer1, answer2, answer3, answer4))

alpha: 50
mse error(ridge): 23.132
ratio: 0.3
mse error(lasso): 26.917
